In [45]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [46]:
import pandas as pd
import numpy as np

installs = pd.read_csv(r'C:\Users\Matias\Documents\UBA\Datos\installs.csv.gzip',compression = 'gzip')

In [47]:
types = {'date':'object','event_id':'int64','ref_type':'category','ref_hash':'int64','application_id':'int64',\
         'attributed':'bool','device_countrycode':'category','device_os_version':'float64','device_brand':'float64',\
         'device_model':'float64','device_city':'float64','session_user_agent':'float64','trans_id':'category',\
         'user_agent':'float64','event_uuid':'object','carrier':'float64','kind':'float64','device_os':'category',\
         'wifi':'bool','connection_type':'category','ip_address':'int64','device_language':'float64'}

cols = ['date','event_id','ref_hash','application_id','attributed','device_brand','device_model'
       ,'event_uuid','carrier','kind','device_os','wifi']

events = pd.read_csv(r'C:\Users\Matias\Documents\UBA\Datos\events.csv.gzip',compression = 'gzip', dtype = types, usecols = cols)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:

#PRE: 
#df['timestamp'] = timestamp de una fecha
#date fecha a comparar
#refhash para filtrar los events
#deltaTime para determinar el margen de aceptacion para exactmatch [Real+]

#POS:
#devuelve 2 dataframes, para checkear si estan vacios usar df.empty

def find_lowest_neighbour_in_time(df,date,refhash,deltaTime=0):
    df = df[df['ref_hash'] == refhash]
    exactmatch=df[(df['date']==date) | 
                   (((date.timestamp()- df['timestamp'])< deltaTime) &
                  ((date.timestamp()- df['timestamp'])> 0))] # quiero solo los eventos previos a date
    lowerneighbour_ind = df[df['timestamp']<date.timestamp()].nlargest(1,'timestamp')
    return lowerneighbour_ind,exactmatch

In [49]:
events['date'] = pd.to_datetime(events['date'])

installs['created'] = pd.to_datetime(installs['created'])
installs['hour'] = installs['created'].apply(lambda x: x.hour)
installs['hour'] = installs['hour'].astype('int8')

installs['day'] = installs['created'].apply(lambda x: x.day)
installs['day'] = installs['day'].astype('int8')

installs['install_value'] = 1

In [50]:
installs['origin_event_uuid'] = None
installs['exact_event_uuid'] = None

In [51]:
events['timestamp'] = events['date'].apply(lambda x: x.timestamp())
events['timestamp'] = events['timestamp'].astype('float32')
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494423 entries, 0 to 2494422
Data columns (total 13 columns):
date              datetime64[ns]
event_id          int64
ref_hash          int64
application_id    int64
attributed        bool
device_brand      float64
device_model      float64
event_uuid        object
carrier           float64
kind              float64
device_os         category
wifi              object
timestamp         float32
dtypes: bool(1), category(1), datetime64[ns](1), float32(1), float64(4), int64(3), object(2)
memory usage: 204.6+ MB


In [52]:
installsByTime = installs.groupby('created').agg({'install_value':sum,'application_id':lambda x: x,'ref_hash':lambda x: x})
installsDevice = installs.groupby('ref_hash').agg({'install_value':sum,'application_id':lambda x: x,'created':lambda x: x})
installsByTime.head()

,install_value,application_id,ref_hash
created,,,
2019-03-05 00:00:38.219,1,5,5924583283197158397
2019-03-05 00:05:35.234,1,16,1740562126672148647
2019-03-05 00:07:25.431,1,16,760677655438420566
2019-03-05 00:07:38.572,1,8,5006759053482448963
2019-03-05 00:10:36.708,1,7,3318538943125451633


In [53]:
#Agrego los eventos de origen y exactos a events

In [55]:
for date in installsByTime.index.tolist():
    neighbour,exact = find_lowest_neighbour_in_time(events,date,installsByTime.loc[date,'ref_hash'],0.1)
    if neighbour.empty == False:
        install = installs[installs['created'] == date]
        value = neighbour.loc[neighbour.index[0],'event_uuid']
        installs.loc[install.index[0],'origin_event_uuid'] = value
    if exact.empty == False:
        install = installs[installs['created'] == date]
        value = exact.loc[exact.index[0],'event_uuid']
        installs.loc[install.index[0],'exact_event_uuid'] = value
        

In [56]:
instsource = installs.dropna(subset = ['origin_event_uuid'])
instexact = installs.dropna(subset = ['exact_event_uuid'])

In [57]:
#lista de apps con evento exacto
instexact['application_id'].unique()

array([ 0,  2,  5,  8,  9, 12, 15, 17, 21, 10, 24, 20,  7, 30, 16,  6, 33,
       29, 34], dtype=int64)

In [58]:
#mi teoria es que las siguientes apps dejan marcada la instalacion implicita como un evento
#para desprobarla debo buscar casos donde:
# 1) existan instalaciones no implicitas que dejen constancia
# 2) existan instalaciones implicitas de las apps en la lista que no dejen constancia del evento
# 3) existan apps fuera de la lista que tengan constancia del evento

#por la forma en la que forme la lista (usando dropna), la 3 esta descartada.

In [59]:
instexact['implicit'].unique()

array([ True])

In [60]:
#Con esto demuestro que unicamente las implicitas dejan constancia

In [61]:
installs_that_break_hypothesis = installs.loc[(installs['application_id'].isin(instexact['application_id'].unique())) &
                  (installs['implicit'] == True) &
                  (installs['exact_event_uuid'].isnull())]

installs_that_break_hypothesis.empty

False

In [62]:
#Resulta que si existen! elimino las apps que rompen mi hipotesis

In [63]:
instexact = instexact[instexact['application_id'].isin(installs_that_break_hypothesis['application_id'].unique()) == False]
instexact['application_id'].unique()

array([ 0,  2,  5,  8, 12, 15, 17, 21, 24, 20,  7, 30, 33, 29, 34],
      dtype=int64)

In [64]:
# estas son las apps id que cumplen con mi hipotesis